In [1]:
!pip install "setfit[absa]"
# or
# !pip install spacy

  Using cached setfit-1.1.0-py3-none-any.whl.metadata (12 kB)
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached huggingface_hub-0.26.3-py3-none-any.whl.metadata (13 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached spacy-3.7.5-cp312-cp312-win_amd64.whl.metadata (27 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.1.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pyarrow-18.1.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-a


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!spacy download en_core_web_lg
!spacy download en_core_web_sm

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     --------------------------------------- 4.5/587.7 MB 24.4 MB/s eta 0:00:24
      -------------------------------------- 7.6/587.7 MB 20.4 MB/s eta 0:00:29
      ------------------------------------- 12.6/587.7 MB 21.9 MB/s eta 0:00:27
     - ------------------------------------ 17.3/587.7 MB 21.8 MB/s eta 0:00:27
     - ------------------------------------ 21.5/587.7 MB 21.6 MB/s eta 0:00:27
     - ------------------------------------ 25.2/587.7 MB 21.2 MB/s eta 0:00:27
     - ------------------------------------ 29.4/587.7 MB 20.9 MB/s eta 0:00:27
     -- ----------------------------------- 33.3/587.7 MB 20.7 MB/s eta 0:00:27
     -- ----------------------------------- 37.2/587.7 MB 20.6 MB/s eta 0:00:27
     -- ----------------------------------- 41.2/587.7 MB 20.4 MB/s eta 0:00:27
     -- ----------------------------------- 44.8/587.7 MB 20.4 MB/s eta 0:00:27
     --- ---------------------------------- 48.


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 578.7 kB/s eta 0:00:22
     --- ------------------------------------ 1.0/12.8 MB 1.1 MB/s eta 0:00:11
     ----- ---------------------------------- 1.8/12.8 MB 1.7 MB/s eta 0:00:07
     --------- ------------------------------ 3.1/12.8 MB 2.5 MB/s eta 0:00:04
     -------------- ------------------------- 4.7/12.8 MB 3.3 MB/s eta 0:00:03
     ------------------- -------------------- 6.3/12.8 MB 3.9 MB/s eta 0:00:02
     ------------------------------ --------- 9.7/12.8 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 6.2 MB/s eta 0:00:01
     -


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Ensure the required version of tokenizers is installed
%pip install tokenizers>=0.20,<0.21
%pip install transformers -U

from setfit import AbsaModel

model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    spacy_model="en_core_web_sm",
)

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.



  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.3-cp312-none-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.20.3-cp312-none-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.2
    Uninstalling transformers-4.42.2:
      Successfully uninstalled transformers-4.42.2


  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\adrie\OneDrive\Documents\GitHub\mmai5400A_assignment3\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adrie\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.micr

In [1]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/tomaarsen/setfit-absa-semeval-restaurants/" + splits["train"])

c:\Users\adrie\OneDrive\Documents\GitHub\mmai5400A_assignment3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pip install datasets

from datasets import load_dataset

# The training/eval dataset must have `text`, `span`, `label`, and `ordinal` columns
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\adrie\OneDrive\Documents\GitHub\mmai5400A_assignment3\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adrie\.cache\huggingface\hub\datasets--tomaarsen--setfit-absa-semeval-restaurants. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get

In [7]:
from setfit import AbsaTrainer, TrainingArguments
from transformers import EarlyStoppingCallback

args = TrainingArguments(
    output_dir="models",
    num_epochs=5,
    use_amp=True,
    batch_size=128,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)

trainer = AbsaTrainer(
    model,
    polarity_model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

NameError: name 'model' is not defined

In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Collecting setfit[absa]\n",
      "  Using cached setfit-1.1.0-py3-none-any.whl.metadata (12 kB)\n",
      "Collecting datasets>=2.15.0 (from setfit[absa])\n",
      "  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)\n",
      "Collecting sentence-transformers>=3 (from sentence-transformers[train]>=3->setfit[absa])\n",
      "  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)\n",
      ...
      "[notice] A new release of pip is available: 24.2 -> 24.3.1\n",
      "[notice] To update, run: python.exe -m pip install --upgrade pip\n"
     ]
    }
   ],
   "source": [
    "!pip install setfit[absa]\n",
    "!pip install spacy\n",
    "!pip install en-core-web-lg==3.7.1\n",
    "!python -m spacy download en_core_web_lg"
   ]
  },
  ...
 ]
}